# Let's explore caching and profiling.

Important words: 

**caching** - Saving a piece of information, usually one that is computationally expensive to fetch, in fast-access memory to hand out to clients who ask for that piece of information. Usually done after the first time the piece of info is fetched. When the result of a function is cached for a specific set of arguments to the function, it's called **memoization**.

**profiling** - Running our code with a tool that tells us how much time and/or memory each portion of our code is using. It is (or should be) the first step in any serious code optimization effort. 

In [1]:
# In REAL LIFE the module you'll import this from is called functools.
# In this version, I deliberately patched lru_cache to give you access to the cache,
# Which you don't get in the "real" version for thread safety reasons.
from chelseas_functools import lru_cache 
import cProfile

Here, I have written a recursive function that calculates the _factorial_ of a number.

Example: 5 factorial, or 5!, is 5 * 4 * 3 * 2 * 1, or 120.

I have decorated it with the `@lru_cache` decorator with a `maxsize` argument of `None`.

[Theis documentation](https://docs.python.org/3/library/functools.html) covers how `@lru_cache` works.

In [2]:
@lru_cache(maxsize=None)
def factorial(num):
    if num == 1:
        return 1
    else:
        return num * factorial(num-1)

Now, I run my factorial function on the number 50 with Python's profiler. [Here is the documentation](https://docs.python.org/3/library/profile.html) on how to read profiler output.

Run the cell below a couple of times and notice how the profiler output changes.

In [4]:
cProfile.run('factorial(51)')

         7 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 chelseas_functools.py:458(_make_key)
        1    0.000    0.000    0.000    0.000 chelseas_functools.py:561(wrapper)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.len}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.000    0.000    0.000    0.000 {method 'get' of 'dict' objects}




The `@lru_cache` decorator adds two methods to the `factorial` function:
    
1. `cache_info`, which gives you four pieces of information about the cache (what are they?)
1. `cache_clear`, which clears the cache so you can run the function "from scratch" again.

In [5]:
factorial.cache_info()

CacheInfo(hits=1, misses=51, maxsize=None, currsize=51)

### OK, so here's where I did something naughty.

I augmented the `@lru_cache` wrapper to give you access to the cache. DO NOT USE the `chelseas_functools` version of `@lru_cache` in any production application. But for our _educational_ purposes, you can here peek inside the cache that the function is keeping:

In [6]:
factorial.cache

{1: 1,
 2: 2,
 3: 6,
 4: 24,
 5: 120,
 6: 720,
 7: 5040,
 8: 40320,
 9: 362880,
 10: 3628800,
 11: 39916800,
 12: 479001600,
 13: 6227020800,
 14: 87178291200,
 15: 1307674368000,
 16: 20922789888000,
 17: 355687428096000,
 18: 6402373705728000,
 19: 121645100408832000,
 20: 2432902008176640000,
 21: 51090942171709440000,
 22: 1124000727777607680000,
 23: 25852016738884976640000,
 24: 620448401733239439360000,
 25: 15511210043330985984000000,
 26: 403291461126605635584000000,
 27: 10888869450418352160768000000,
 28: 304888344611713860501504000000,
 29: 8841761993739701954543616000000,
 30: 265252859812191058636308480000000,
 31: 8222838654177922817725562880000000,
 32: 263130836933693530167218012160000000,
 33: 8683317618811886495518194401280000000,
 34: 295232799039604140847618609643520000000,
 35: 10333147966386144929666651337523200000000,
 36: 371993326789901217467999448150835200000000,
 37: 13763753091226345046315979581580902400000000,
 38: 52302261746660111176000722410007429120000

How many items are in this cache? Is that how many items you expected this cache to have in there?

Does this cache **remind** you of any in-class exercises we have done?

## Let's try out @lru_cache with a maxsize.

So far in this exercise, we have had the `@lru_cache`'s `maxsize` argument set to `None`. This means that the cache increases in size _indefinitely_. Python has another decorator called `@cache` that has the same behavior as `@lru_cache` with a `maxsize` set to `None`.

#### Questions for you:

1. Based on the documentation you read, what is the default value for `maxsize` in `@lru_cache`?
2. What are the units of `maxsize`? What is it saving that many of?

Let's set the `@lru_cache`'s `maxsize` to something else: ten, for example. 

In [7]:
@lru_cache(maxsize=10)
def factorial(num):
    if num == 1:
        return 1
    else:
        return num * factorial(num-1)

Now, let's run our wrapped function on a number (I have deliberately chosen a small number to make the following examples easier to visually inspect).

In [8]:
factorial(3)

6

**Here's where things get wild**: observe below how the data structure used for the cache _changes_ when the cache has a `maxsize`. 

What data structure is that?
Why do you think it's getting used here? 

**Hint #1:** When `@lru_cache` has a limited size, how does it decide which items to keep and which items to get rid of? Does the data structure we saw above support that use case?

**Hint #2:** you can go _look_ at the actual, _real_ implementation of `lru_cache` [right here](https://github.com/python/cpython/blob/main/Lib/functools.py)—or even my copied version in the `chelseas_functools` module in this directory, which is _almost exactly_ the same as the real implementation. Search in the file for `def _lru_cache_wrapper`, which (shocking!) defines the `lru_cache` wrapper. See how that function _switches_ how it implements the cache based on the `maxsize` argument?

Here's what the cache looks like when it has a `maxsize`: 

In [9]:
factorial.cache

{1: [[[[[...], [...], 2, 2], [...], 3, 6], [...], None, None],
  [[...], [[...], [[...], [...], None, None], 3, 6], 2, 2],
  1,
  1],
 2: [[[[[...], [...], 3, 6], [...], None, None], [...], 1, 1],
  [[...], [[...], [[...], [...], 1, 1], None, None], 3, 6],
  2,
  2],
 3: [[[[[...], [...], None, None], [...], 1, 1], [...], 2, 2],
  [[...], [[...], [[...], [...], 2, 2], 1, 1], None, None],
  3,
  6]}

In [ ]:
id(factorial.cache[1][1]) == id(factorial.cache[3][0])